In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
from experiments.founder_rank.helpers import *
import experiments.founder_rank.metrics as metric

In [4]:
import experiments.founder_rank.data.baseline as baseline
import experiments.founder_rank.data.random as random
import experiments.founder_rank.data.naive as naive

In [5]:
datasets = get_datasets([baseline, random, naive])

In [6]:
metrics = metric.Metrics(datasets['baseline'])

In [8]:
print(metrics.idcg)
for_each_dataset(datasets, metrics.ndcg)

1.452553083084042e+168
baseline: 1.0
random: 0.2704474479896671
naive: 0.4482823680815219


In [11]:
for n in [5, 10, 20, 50]:
  print('n = {n}'.format(n=n))
  for_each_dataset(datasets, lambda a: metrics.precision_at(n, a))

n = 5
baseline: 1.0
random: 0.2
naive: 0.2
n = 10
baseline: 1.0
random: 0.1
naive: 0.3
n = 20
baseline: 1.0
random: 0.05
naive: 0.2
n = 50
baseline: 1.0
random: 0.04
naive: 0.38


In [12]:
for_each_dataset(datasets, metrics.tau)

baseline: 1.0


random: 0.006891758846354318


naive: 0.4845401954917215


In [13]:
for_each_dataset(datasets, metrics.rho)

baseline: 1.0
random: 0.010010892448288122
naive: 0.6734281623160935


In [14]:
for_each_dataset(datasets, metrics.rmse)

baseline: 0.0
random: 0.25540572915869914
naive: 0.08274775113989331


In [15]:
for_each_dataset(datasets, metrics.mae)

baseline: 0.0
random: 0.2344563094356908
naive: 0.06373322143862112
